In [13]:
p = 3380659
F = GF(p)
R.<x> = PolynomialRing(F)
f = R([831453, 1848930, 2461019, 1710912, 1959961, 467855, 55393, 249259, 916975, 183633, 59966, 562171, 189567, 839900, 2656364, 3371938, 0, 1])
Roots = []

def probability_algorithm():
    g = gcd(f, x^p-x)
    n = g.degree()
    if n == 0:
        print('deg(g) = 0. Корней нет')
        return 0
    elif n == 1:
        print('deg(g) = 1')
        return -g[0]
    elif n == p:
        print('deg(g) - p. Корни - все элементы Fp')
        return 'all'

    decomposition(g)
 
    print('f(x) = ', f)
    print('Probability algorithm: ', Roots)
    print('Sage: ', f.roots())


def decomposition(g):
    if g.degree () ==0:
        return 0
    
    while True:
        d = gcd((x + randint(1, p))^((p - 1) / 2) - 1,g)

        if d != R(1) or d != g:
            break

    n = d.degree()

    if n == 1:
        b = -d.coefficients()[0]
        Roots.append(b)
        g = g // (x - b)
        return decomposition(g)
    elif n == g.degree() - 1:
        t = g // d
        b = -t.coefficients()[0]
        Roots.append(b)
        return decomposition(d)
    else:
        return decomposition(d), decomposition(g // d)

probability_algorithm()

f(x) =  x^17 + 3371938*x^15 + 2656364*x^14 + 839900*x^13 + 189567*x^12 + 562171*x^11 + 59966*x^10 + 183633*x^9 + 916975*x^8 + 249259*x^7 + 55393*x^6 + 467855*x^5 + 1959961*x^4 + 1710912*x^3 + 2461019*x^2 + 1848930*x + 831453
Probability algorithm:  [593791, 2998547]
Sage:  [(2998547, 1), (593791, 1)]


In [14]:
def create_matrix_B1(n, R, q, f):
    x = R.gen()
    a = R(1)
    B = [0] * n
    for i in range(n):
        B[i] = [0] * n
    
    for i in range(n):
        for j in range(n):
            B[j][i] = a[j]
            if i == j:
                B[i][i] -= 1
        a = (a*x**q)%f
    return Matrix(GF(q), B)

def print_factor(U):
    for fnom in U:
        print ('(' + str(fnom) + ')')

def Berlikamp(q, coeffs):
    K = GF(q)
    R.<x> = PolynomialRing(K)
    f = R(coeffs)

    U_prev = []
    f_der = f.derivative()
   
    if f_der == 0:
        return Berlikamp(q, f.list()[0::q])

    d = gcd(f, f_der)
    if d.degree() != 0:
        f = R(f/d)
    n = f.degree()

    B1 = create_matrix_B1(n, R, q, f)

    if B1.rank() == n - 1:
        print('Многочлен неприводимый')
        return

    e = B1.right_kernel().basis_matrix().rows()

    r = len(e)
    i = 1
    l = 0
    U = [R(0)]*r
    U[l] = f

    while l != (r - 1) and i != r:

        h = R(0)
        for j in range(n):
            h += e[i][j]*x**j

        for j in range(l + 1):

            s = 0
            gx = []
            for c in range(q):
                d = gcd(U[j], h - R(c))
                if (d != R(1)) and (d not in gx):
                    gx.append(d)
                    s += 1

            U[j] = gx[0]
            t = 1
            while t < s:
                U[l + t] = gx[t]
                t += 1
            l = l+s-1

            if l == r - 1:
                break

        i += 1
    return U

q = 2050079
R.<x> = PolynomialRing(Zmod(q))

init_coeffs = [273649, 222532, 1696631, 1009691, 1645530, 1850770, 1639985, 796104, 48493, 27932, 607298, 1344201, 780409, 1931169, 124879, 2024971, 1788, 2050014, 1]

f0 = R(init_coeffs)
U = Berlikamp(q, init_coeffs)

print('Sage:')
print(f0.factor())
print()
print('Berlikamp:')
print_factor(U)

Sage:
(x^2 + 2050061*x + 82)^2 * (x^3 + 2050065*x^2 + 65*x + 2049980)^2 * (x^8 + 2050078*x^7 + 2049981*x^6 + 812*x^5 + 2046999*x^4 + 6608*x^3 + 2041791*x^2 + 5696*x + 2048416)

Berlikamp:
(x^3 + 2050065*x^2 + 65*x + 2049980)
(x^2 + 2050061*x + 82)
(x^8 + 2050078*x^7 + 2049981*x^6 + 812*x^5 + 2046999*x^4 + 6608*x^3 + 2041791*x^2 + 5696*x + 2048416)
